START

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os, glob

candidates = glob.glob("/content/drive/MyDrive/**/*.zip", recursive=True)
# Show only zip files that look like flickr8k
[f for f in candidates if "flickr" in f.lower() or "flickr8k" in f.lower()][:20]


['/content/drive/MyDrive/flickr8k.zip']

In [3]:
ZIP_PATH = "/content/drive/MyDrive/flickr8k.zip"
OUT_DIR = "/content/data/flickr8k"

!mkdir -p "$OUT_DIR"
!unzip -q "$ZIP_PATH" -d "$OUT_DIR"
print("Unzipped to:", OUT_DIR)


Unzipped to: /content/data/flickr8k


In [5]:
import os

OUT_DIR = "/content/data/flickr8k"
print("Top-level folders/files inside OUT_DIR:")
print(os.listdir(OUT_DIR)[:50])


Top-level folders/files inside OUT_DIR:
['Images', 'captions.txt']


In [6]:
import os

OUT_DIR = "/content/data/flickr8k"
images_dir = os.path.join(OUT_DIR, "Images")
captions_file = os.path.join(OUT_DIR, "captions.txt")

print("Images dir exists:", os.path.isdir(images_dir))
print("Num images:", len([f for f in os.listdir(images_dir) if f.lower().endswith(".jpg")]))

print("Captions file exists:", os.path.isfile(captions_file))
with open(captions_file, "r") as f:
    for _ in range(5):
        print(f.readline().strip())


Images dir exists: True
Num images: 8091
Captions file exists: True
image,caption
1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg,A girl going into a wooden building .
1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .


MOST IMORTANT CELL FOR EXECUTION

In [11]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)

print("Model loaded on", device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Model loaded on cuda


REBUILDING

In [4]:
import transformers, tokenizers
print("transformers:", transformers.__version__)
print("tokenizers:", tokenizers.__version__)


transformers: 4.47.1
tokenizers: 0.21.4


In [5]:
import os, random
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np

from transformers import BlipProcessor, BlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

print("Model loaded")


Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded ✅


In [6]:
OUT_DIR = "/content/data/flickr8k"
images_dir = os.path.join(OUT_DIR, "Images")
captions_file = os.path.join(OUT_DIR, "captions.txt")

print("Images:", len([f for f in os.listdir(images_dir) if f.lower().endswith(".jpg")]))
print("Captions file exists:", os.path.isfile(captions_file))


Images: 8091
Captions file exists: True


In [7]:
import pandas as pd
import numpy as np

rows = []
with open(captions_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if "," in line:
            img, cap = line.split(",", 1)
        else:
            continue
        img, cap = img.strip(), cap.strip()
        if img.lower() == "image" and cap.lower() == "caption":
            continue
        rows.append({"image": img, "caption": cap})

df = pd.DataFrame(rows)

# split by image (80/10/10)
unique_images = df["image"].unique()
rng = np.random.default_rng(42)
rng.shuffle(unique_images)

n = len(unique_images)
train_end = int(0.8 * n)
val_end   = int(0.9 * n)

train_imgs = set(unique_images[:train_end])
val_imgs   = set(unique_images[train_end:val_end])
test_imgs  = set(unique_images[val_end:])

def assign_split(img):
    if img in train_imgs: return "train"
    if img in val_imgs: return "val"
    return "test"

df["split"] = df["image"].apply(assign_split)

print("Rows per split:")
print(df["split"].value_counts())
print("\nUnique images per split:")
print(df.groupby("split")["image"].nunique())


Rows per split:
split
train    32360
test      4050
val       4045
Name: count, dtype: int64

Unique images per split:
split
test      810
train    6472
val       809
Name: image, dtype: int64


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
from PIL import Image

MAX_TEXT_LEN = 32

class FlickrCaptionDataset(Dataset):
    def __init__(self, df, split):
        d = df[df["split"] == split].reset_index(drop=True)
        self.grouped = d.groupby("image")["caption"].apply(list).reset_index()

    def __len__(self):
        return len(self.grouped)

    def __getitem__(self, idx):
        row = self.grouped.iloc[idx]
        img_name = row["image"]
        caption = random.choice(row["caption"])

        img_path = os.path.join(images_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        enc = processor(
            images=image,
            text=caption,
            padding="max_length",
            truncation=True,
            max_length=MAX_TEXT_LEN,
            return_tensors="pt",
        )

        item = {k: v.squeeze(0) for k, v in enc.items()}

        labels = item["input_ids"].clone()
        labels[labels == processor.tokenizer.pad_token_id] = -100
        item["labels"] = labels
        return item

def collate_fn(batch):
    keys = batch[0].keys()
    return {k: torch.stack([b[k] for b in batch]) for k in keys}

train_dataset = FlickrCaptionDataset(df, "train")
val_dataset   = FlickrCaptionDataset(df, "val")
test_dataset  = FlickrCaptionDataset(df, "test")

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=0)

print("Train batches:", len(train_loader), "Val batches:", len(val_loader))


Train batches: 1618 Val batches: 203


In [9]:
from torch.optim import AdamW

model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)

def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total = 0.0
    n = 0

    for step, batch in enumerate(loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.set_grad_enabled(train):
            out = model(**batch)
            loss = out.loss

            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        total += loss.item()
        n += 1

        if train and step % 100 == 0:
            print(f"step {step}/{len(loader)}  loss={loss.item():.4f}")

    return total / max(n, 1)

train_loss = run_epoch(train_loader, train=True)
val_loss   = run_epoch(val_loader, train=False)
print(f"\nDONE   train_loss={train_loss:.4f}  val_loss={val_loss:.4f}")


step 0/1618  loss=2.8844
step 100/1618  loss=2.4189
step 200/1618  loss=2.3037
step 300/1618  loss=1.9713
step 400/1618  loss=3.0806
step 500/1618  loss=1.7373
step 600/1618  loss=1.9180
step 700/1618  loss=1.7355
step 800/1618  loss=2.0779
step 900/1618  loss=1.5240
step 1000/1618  loss=2.0413
step 1100/1618  loss=2.2741
step 1200/1618  loss=2.2414
step 1300/1618  loss=2.8511
step 1400/1618  loss=2.5794
step 1500/1618  loss=2.2575
step 1600/1618  loss=1.5304

DONE ✅  train_loss=2.3334  val_loss=2.2349


In [10]:
SAVE_DIR = "/content/blip_flickr8k_finetuned"
model.save_pretrained(SAVE_DIR)
processor.save_pretrained(SAVE_DIR)
print("Saved to:", SAVE_DIR)


Saved to: /content/blip_flickr8k_finetuned


In [11]:
import pandas as pd
import random
from PIL import Image

model.eval()

test_grouped = df[df["split"]=="test"].groupby("image")["caption"].apply(list).reset_index()
sample_rows = test_grouped.sample(10, random_state=42).reset_index(drop=True)

results_samples = []

for i, row in sample_rows.iterrows():
    img_name = row["image"]
    refs = row["caption"][:3]  # show 3 reference captions
    img_path = os.path.join(images_dir, img_name)
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_length=20, num_beams=5)
    pred = processor.decode(out[0], skip_special_tokens=True)

    results_samples.append({
        "image": img_name,
        "prediction": pred,
        "ref1": refs[0],
        "ref2": refs[1] if len(refs)>1 else "",
        "ref3": refs[2] if len(refs)>2 else ""
    })

samples_df = pd.DataFrame(results_samples)
samples_df


,image,prediction,ref1,ref2,ref3
0,2621771656_09a620da6d.jpg,a group of people stand in front of a bridge.,A crowd of people are standing behind a railin...,A group of people wait behind a barricade and ...,A group of people wait behind a metal fence .
1,3688839836_ba5e4c24fc.jpg,a woman in a red shirt holds up a sign.,A woman is holding out a peace sign during a p...,A woman wearing a red scarf raises her hand as...,People in orange shirts marching down street w...
2,2557972410_6925fe695c.jpg,a man in a helmet is playing hockey.,A goalie is guarding the goal .,Kids in helmets play hockey outside .,Several players playing a game of hockey outsi...
3,2759596272_e0ce0a965a.jpg,a little girl in a striped shirt is swinging o...,a girl wearing pink shoes jumps off a swing in...,A girl with blonde hair is jumping off a tire ...,A young girl jumps off her swing in front of b...
4,3415646718_f9f4e23a66.jpg,a man in a cowboy outfit is riding a bull.,A man is riding a bull at a rodeo setting with...,A man riding a bucking bull at a rodeo .,bull riding with people watching
5,862187579_9faf4a51e0.jpg,a man in a blue jacket and glasses looks out o...,A man wearing glasses is staring off into space .,A man with glasses is standing in front of a b...,A man with sunglasses looking straight ahead .
6,2460823604_7f6f786b1c.jpg,a boy in a yellow shirt and white pants is pla...,A boy baseball player in a green and yellow je...,A boy chasing a baseball .,A little boy playing baseball goes for the bal...
7,3534824784_7133119316.jpg,a man and a woman pose for a picture.,A girl leaning against a woman 's shoulder whi...,A mother with a child sitting on her lap and b...,A woman and a child sit together and smile .
8,2877159456_ea4a46b0d2.jpg,a man in a blue shirt and a man in a blue shir...,A man and a woman with three bikes and a silve...,A man and woman with bicycles stand by a silve...,"""A man , woman and three bicycles are resting ..."
9,3476451861_5b9c9ce191.jpg,a man in a white shirt is doing a trick on a b...,A bicyclist is performing a trick on a tall tr...,A person is riding a bike on a tree .,A person riding a bike with blue wheels on a t...


In [12]:
!pip -q install nltk


In [13]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

smooth = SmoothingFunction().method1
model.eval()

# Evaluate on 200 test images for speed
test_subset = test_grouped.sample(n=min(200, len(test_grouped)), random_state=42).reset_index(drop=True)

predictions = []
references = []

for _, row in test_subset.iterrows():
    img_name = row["image"]
    ref_caps = row["caption"]
    img_path = os.path.join(images_dir, img_name)
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_length=20, num_beams=5)
    pred = processor.decode(out[0], skip_special_tokens=True)

    # BLEU expects tokenized refs/preds
    predictions.append(pred.split())
    references.append([r.split() for r in ref_caps])

bleu_score = corpus_bleu(references, predictions, smoothing_function=smooth)
print("BLEU (200 test images):", bleu_score)


BLEU (200 test images): 0.17805705554978052


In [14]:
train_loss2 = run_epoch(train_loader, train=True)
val_loss2   = run_epoch(val_loader, train=False)
print(f"IMPROVED  train_loss={train_loss2:.4f}  val_loss={val_loss2:.4f}")


step 0/1618  loss=2.1682
step 100/1618  loss=2.3992
step 200/1618  loss=1.6267
step 300/1618  loss=2.4543
step 400/1618  loss=1.9511
step 500/1618  loss=1.7267
step 600/1618  loss=2.3387
step 700/1618  loss=1.3729
step 800/1618  loss=1.6781
step 900/1618  loss=2.1251
step 1000/1618  loss=1.9999
step 1100/1618  loss=2.1023
step 1200/1618  loss=2.1102
step 1300/1618  loss=1.8385
step 1400/1618  loss=2.0049
step 1500/1618  loss=2.6297
step 1600/1618  loss=2.5368
IMPROVED ✅ train_loss=2.0635  val_loss=2.2337


In [15]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from PIL import Image
import os
import numpy as np

smooth = SmoothingFunction().method1
model.eval()

# Recreate grouped test set (if not already)
test_grouped = df[df["split"]=="test"].groupby("image")["caption"].apply(list).reset_index()

# Evaluate on 200 test images for speed
test_subset = test_grouped.sample(n=min(200, len(test_grouped)), random_state=42).reset_index(drop=True)

predictions = []
references = []

for _, row in test_subset.iterrows():
    img_name = row["image"]
    ref_caps = row["caption"]
    img_path = os.path.join(images_dir, img_name)
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_length=20, num_beams=5)
    pred = processor.decode(out[0], skip_special_tokens=True)

    predictions.append(pred.split())
    references.append([r.split() for r in ref_caps])

bleu_score_2 = corpus_bleu(references, predictions, smoothing_function=smooth)
print("BLEU (200 test images) AFTER 2 epochs:", bleu_score_2)


BLEU (200 test images) AFTER 2 epochs: 0.1944406075573781


In [16]:
import pandas as pd

results_table = pd.DataFrame([
    {"Run": "Epoch 1 (baseline)", "Train Loss": 2.3334, "Val Loss": 2.2349, "BLEU@200": 0.17805705554978052},
    {"Run": "Epoch 2 (improved)", "Train Loss": 2.0635, "Val Loss": 2.2337, "BLEU@200": 0.1944406075573781},
])

results_table


,Run,Train Loss,Val Loss,BLEU@200
0,Epoch 1 (baseline),2.3334,2.2349,0.178057
1,Epoch 2 (improved),2.0635,2.2337,0.194441


In [17]:
results_csv_path = "/content/drive/MyDrive/blip_results_table_epoch2.csv"
results_table.to_csv(results_csv_path, index=False)
print(" Saved:", results_csv_path)


✅ Saved: /content/drive/MyDrive/blip_results_table_epoch2.csv


In [18]:
import pandas as pd
from PIL import Image
import os
import torch

model.eval()

test_grouped = df[df["split"]=="test"].groupby("image")["caption"].apply(list).reset_index()
sample_rows = test_grouped.sample(10, random_state=7).reset_index(drop=True)

qual_rows = []

for _, row in sample_rows.iterrows():
    img_name = row["image"]
    refs = row["caption"][:3]
    img_path = os.path.join(images_dir, img_name)
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_length=20, num_beams=5)
    pred = processor.decode(out[0], skip_special_tokens=True)

    qual_rows.append({
        "image": img_name,
        "prediction_epoch2": pred,
        "ref1": refs[0],
        "ref2": refs[1] if len(refs)>1 else "",
        "ref3": refs[2] if len(refs)>2 else "",
    })

qual_epoch2 = pd.DataFrame(qual_rows)
qual_path = "/content/drive/MyDrive/blip_qualitative_epoch2.csv"
qual_epoch2.to_csv(qual_path, index=False)
print(" Saved:", qual_path)

qual_epoch2.head(5)


✅ Saved: /content/drive/MyDrive/blip_qualitative_epoch2.csv


,image,prediction_epoch2,ref1,ref2,ref3
0,3242007318_01e82171aa.jpg,a man and a woman stand in front of a restaurant.,Two Asian girls are standing outside a red bri...,Two asian ladies posing for a picture .,Two women standing in front of a large brick b...
1,697778778_b52090709d.jpg,a man climbing a rock face.,Three climbers on rocks near waterfall .,Three men climb a natural waterfall rock face .,Three men in matching outfits climb a rock wal...
2,361183669_52be9662b9.jpg,a group of people on a boat in the water.,people are getting ready to set sail on a boat,People on a boat carrying equipment .,Three men prepare to set sail on a sailboat .
3,2707873672_15e6b5d54b.jpg,a man in a brown shirt is doing a trick on a s...,Two boys wearing brown shirts tug at a rope on...,Two boys with red hair and glasses stand on a ...,Two redheaded boys in brown shirts stand on a ...
4,2327088022_478dbd2c17.jpg,a man in a red helmet is climbing a snowy moun...,A man climbing an icy glacier with the assista...,A man donning an orange helmet is climbing up ...,A man is chiseling into ice .


In [19]:
train_loss3 = run_epoch(train_loader, train=True)
val_loss3   = run_epoch(val_loader, train=False)
print(f"EPOCH 3  train_loss={train_loss3:.4f}  val_loss={val_loss3:.4f}")


step 0/1618  loss=2.5359
step 100/1618  loss=2.1638
step 200/1618  loss=1.7975
step 300/1618  loss=2.3915
step 400/1618  loss=1.5108
step 500/1618  loss=2.5305
step 600/1618  loss=1.4045
step 700/1618  loss=1.6812
step 800/1618  loss=1.6940
step 900/1618  loss=1.7379
step 1000/1618  loss=1.6497
step 1100/1618  loss=1.9033
step 1200/1618  loss=2.5863
step 1300/1618  loss=1.7545
step 1400/1618  loss=1.4108
step 1500/1618  loss=2.5766
step 1600/1618  loss=1.1824
EPOCH 3 ✅ train_loss=1.9360  val_loss=2.1495


In [20]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from PIL import Image
import os
import torch

smooth = SmoothingFunction().method1
model.eval()

test_grouped = df[df["split"]=="test"].groupby("image")["caption"].apply(list).reset_index()
test_subset = test_grouped.sample(n=min(200, len(test_grouped)), random_state=42).reset_index(drop=True)

predictions = []
references = []

for _, row in test_subset.iterrows():
    img_name = row["image"]
    ref_caps = row["caption"]
    img_path = os.path.join(images_dir, img_name)
    image = Image.open(img_path).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_length=20, num_beams=5)
    pred = processor.decode(out[0], skip_special_tokens=True)

    predictions.append(pred.split())
    references.append([r.split() for r in ref_caps])

bleu_score_3 = corpus_bleu(references, predictions, smoothing_function=smooth)
print("BLEU (200 test images) AFTER 3 epochs:", bleu_score_3)


BLEU (200 test images) AFTER 3 epochs: 0.1588798918541064


In [21]:
import pandas as pd

results_table_3 = pd.DataFrame([
    {"Run": "Epoch 1 (baseline)", "Train Loss": 2.3334, "Val Loss": 2.2349, "BLEU@200": 0.17805705554978052},
    {"Run": "Epoch 2 (improved)", "Train Loss": 2.0635, "Val Loss": 2.2337, "BLEU@200": 0.1944406075573781},
    {"Run": "Epoch 3 (final)", "Train Loss": float(f"{train_loss3:.4f}"), "Val Loss": float(f"{val_loss3:.4f}"), "BLEU@200": float(bleu_score_3)},
])

results_table_3


,Run,Train Loss,Val Loss,BLEU@200
0,Epoch 1 (baseline),2.3334,2.2349,0.178057
1,Epoch 2 (improved),2.0635,2.2337,0.194441
2,Epoch 3 (final),1.9360,2.1495,0.158880


In [22]:
results_csv_path = "/content/drive/MyDrive/blip_results_table_epoch3.csv"
results_table_3.to_csv(results_csv_path, index=False)
print(" Saved:", results_csv_path)


✅ Saved: /content/drive/MyDrive/blip_results_table_epoch3.csv


In [23]:
FINAL_DIR = "/content/drive/MyDrive/blip_flickr8k_finetuned_epoch3"
model.save_pretrained(FINAL_DIR)
processor.save_pretrained(FINAL_DIR)
print(" Saved final model to:", FINAL_DIR)


✅ Saved final model to: /content/drive/MyDrive/blip_flickr8k_finetuned_epoch3


END OF FILE